In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
import os
from pathlib import Path
new_version_path = Path(os.getenv('VNPY2.0.8'))
sys.path.insert(0, str(new_version_path))

import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
pd.set_option('display.float_format', '{:.2f}'.format)

from datetime import datetime
from research_backtest import (run_backtest_for_portfolio, run_portfolio, ResearchBacktestingEngine,
                               get_future_trade_date_index, get_hot_start, params_to_str)
from utility import get_output_folder, get_output_path


def show_portfolio(df, capital=1000000):
    engine = ResearchBacktestingEngine()
    engine.capital = capital
    res = engine.calculate_statistics(df)
    engine.show_chart(df)
    return res

In [ ]:
# commodity_list = [
#     "cu", "al", "ag", "rb", "bu", "ru", "sp",
#     "y", "a", "b", "p", "c", "cs", "jd", "pp",
#     "SR", "CF", "ZC", "FG", "TA", "MA", "RM", "SF", "SM"
# ]

# commodity_list = [
#     "cu", "al", "zn"
# ]

commodity_list = [
    "cu", "al", "zn", "pb", "ni", "sn", "au", "ag", "rb", "hc", "bu", "ru", "sp",
    "m", "y", "a", "b", "p", "c", "cs", "jd", "l", "v", "pp", "j", "jm", "i",
    "SR", "CF", "ZC", "FG", "TA", "MA", "OI", "RM", "SF", "SM"
]

# commodity_list = [
#     "cu", "al", "ag", "rb", "bu", "ru", "sp",
#     "m", "y", "a", "c","jd", "pp", "j", "i",
#     "SR", "CF", "ZC", "FG", "TA", "MA", "SF"
# ]

commodity_list = ["SM", "rb", "sp", "bu", "ag", "SR", "TA", "jd", "a", "AP", "ru", "cu", "ZC", "CF", "pp", "p", "MA"]

# commodity_list = ["rb", "SM", "MA", "p", "bu", "jd", "RM", "pp", "ZC"]
# commodity_list = ["al", "cu", "rb", "zn", "i", "j", "m", "jm"]


start = datetime(2014, 1, 1)
end = datetime(2020, 3, 1)
capital = 1000000
note_str = 'double_ma_daily'
# commodity_list = [commodity for commodity in commodity_list if get_hot_start(commodity) < start]
print(commodity_list)

strategy_name = 'double_ma'
setting = {
    'fast_window': 10,
    'slow_window': 20,
}


# setting = {
#     'entry_window': 80,
#     'exit_window': 40,
#     'stop_multiple': 10,
#     'risk_percent': 0.002
# }

# strategy_name = 'boll_fluid'
# setting = {
#     'boll_window': 80,
#     'boll_dev': 2,
# }

In [ ]:
df = run_portfolio(commodity_list,
                   strategy_name,
                   setting,
                   start,
                   end,
                   capital,
                   note_str,
                   empty_cost=False,
                   cost_multiple=1.8,
                   interval='d')

In [ ]:
%matplotlib notebook
show_portfolio(df, capital)

### 数据校验

In [ ]:
folder = get_output_folder('portfolio', f'{strategy_name}-{note_str}', 'sub_result')
files = list(folder.glob(f'*.csv'))

res_list = []
for file in files:
    commodity = file.name.replace('.csv', '')
    df = pd.read_csv(file, index_col=0)
    stats_series = df.sum()
    stats_series['commodity'] = commodity
    stats_series['day_count'] = len(df)
    res_list.append(stats_series)

df = pd.DataFrame(res_list)
df.sort_values(by='net_pnl', ascending=False)

In [ ]:
df.sum()

In [ ]:
portfolio_file_name = f'{strategy_name}@{params_to_str(setting)}@{note_str}.csv'
file_path = get_output_path(portfolio_file_name, 'portfolio', f'{strategy_name}-{note_str}')
print(file_path)

sum_df = pd.read_csv(file_path, index_col=0, parse_dates=[0])
# sum_df.info()
show_portfolio(sum_df)

In [ ]:
commodity = 'AP'
setting = {
    'entry_window': 60,
    'exit_window': 20,
}
start = datetime(2019, 9, 1)
end = datetime(2020, 3, 1)
capital = 1000000

df = run_backtest_for_portfolio('turtle_exit_ma', setting, commodity, '1h', start, end, capital)
df2 = df.copy()
show_portfolio(df)

In [ ]:
dates = get_future_trade_date_index(start, end)
df2 = df2.reindex(dates, fill_value=0)
show_portfolio(df2)